<a href="https://colab.research.google.com/github/ibonetc/LLM_EPM/blob/main/LLM_FineTuning_sql.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ejemplo de Fine Tuning para lenguaje SQL

In [ ]:
%pip install transformers peft accelerate bitsandbytes datasets -q -U

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
petastorm 0.12.1 requires pyspark>=2.1.0, which is not installed.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [ ]:
dbutils.library.restartPython()

## Cargar datos de hugging face

Base de datos **sql-create-context**, que tiene instrucciones para creación de diferentes códigos sql

In [ ]:
import pandas as pd
from datasets import load_dataset
from IPython.display import HTML, display
import os
from transformers import AutoTokenizer

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

2024-11-07 01:37:39.544218: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
token_file_path = '/Volumes/universidad_eia/default/isis/token_HF.txt'

with open(token_file_path, 'r') as file:
    token = file.read().strip()

os.environ["HUGGINGFACE_HUB_TOKEN"] = token

In [ ]:
dataset_name = "b-mc2/sql-create-context"
dataset = load_dataset(dataset_name, split="train")

/databricks/python_shell/dbruntime/huggingface_patches/datasets.py:45: UserWarning: The cache_dir for this dataset is /root/.cache, which is not a persistent path.Therefore, if/when the cluster restarts, the downloaded dataset will be lost.The persistent storage options for this workspace/cluster config are: [DBFS, UC Volumes].Please update either `cache_dir` or the environment variable `HF_DATASETS_CACHE`to be under one of the following root directories: ['/dbfs/', '/Volumes/']
  warnings.warn(warning_message)


README.md:   0%|          | 0.00/4.43k [00:00<?, ?B/s]

/databricks/python_shell/dbruntime/huggingface_patches/datasets.py:14: UserWarning: During large dataset downloads, there could be multiple progress bar widgets that can cause performance issues for your notebook or browser. To avoid these issues, use `datasets.utils.logging.disable_progress_bar()` to turn off the progress bars.
  warnings.warn(


sql_create_context_v4.json:   0%|          | 0.00/21.8M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/78577 [00:00<?, ? examples/s]

In [ ]:
df = pd.DataFrame(dataset)

In [ ]:
df.head(5)

,answer,question,context
0,SELECT COUNT(*) FROM head WHERE age > 56,How many heads of the departments are older than 56 ?,CREATE TABLE head (age INTEGER)
1,"SELECT name, born_state, age FROM head ORDER BY age","List the name, born state and age of the heads of departments ordered by age.","CREATE TABLE head (name VARCHAR, born_state VARCHAR, age VARCHAR)"
2,"SELECT creation, name, budget_in_billions FROM department","List the creation year, name and budget of each department.","CREATE TABLE department (creation VARCHAR, name VARCHAR, budget_in_billions VARCHAR)"
3,"SELECT MAX(budget_in_billions), MIN(budget_in_billions) FROM department",What are the maximum and minimum budget of the departments?,CREATE TABLE department (budget_in_billions INTEGER)
4,SELECT AVG(num_employees) FROM department WHERE ranking BETWEEN 10 AND 15,What is the average number of employees of the departments whose rank is between 10 and 15?,"CREATE TABLE department (num_employees INTEGER, ranking INTEGER)"


In [ ]:
pd.set_option("display.max_colwidth", None)
pd.set_option("display.width", None)
pd.set_option("display.max_rows", None)

In [ ]:
df.head(5)

,answer,question,context
0,SELECT COUNT(*) FROM head WHERE age > 56,How many heads of the departments are older than 56 ?,CREATE TABLE head (age INTEGER)
1,"SELECT name, born_state, age FROM head ORDER BY age","List the name, born state and age of the heads of departments ordered by age.","CREATE TABLE head (name VARCHAR, born_state VARCHAR, age VARCHAR)"
2,"SELECT creation, name, budget_in_billions FROM department","List the creation year, name and budget of each department.","CREATE TABLE department (creation VARCHAR, name VARCHAR, budget_in_billions VARCHAR)"
3,"SELECT MAX(budget_in_billions), MIN(budget_in_billions) FROM department",What are the maximum and minimum budget of the departments?,CREATE TABLE department (budget_in_billions INTEGER)
4,SELECT AVG(num_employees) FROM department WHERE ranking BETWEEN 10 AND 15,What is the average number of employees of the departments whose rank is between 10 and 15?,"CREATE TABLE department (num_employees INTEGER, ranking INTEGER)"


## Dividir los datos en entrenamiento y prueba

In [ ]:
split_dataset = dataset.train_test_split(test_size=0.2, seed=42)
train_dataset = split_dataset["train"]
test_dataset = split_dataset["test"]

print(f"Base de entrenamiento: {len(train_dataset)} pares de texto a SQL ")
print(f"Base de prueba: {len(test_dataset)} pares texto a SQL ")

Base de entrenamiento: 62861 pares de texto a SQL 
Base de prueba: 15716 pares texto a SQL 


## Definir el prompt

In [ ]:
PROMPT_TEMPLATE = """You are a powerful text-to-SQL model. Given the SQL tables and natural language question, your job is to write SQL query that answers the question.

### Table:
{context}

### Question:
{question}

### Response:
{output}"""

In [ ]:
def apply_prompt_template(row):
    prompt = PROMPT_TEMPLATE.format(
        question=row["question"],
        context=row["context"],
        output=row["answer"],
    )
    return {"prompt": prompt}

In [ ]:
train_dataset = train_dataset.map(apply_prompt_template)

Map:   0%|          | 0/62861 [00:00<?, ? examples/s]

In [ ]:
df_train=train_dataset.to_pandas()

In [ ]:
df_train.head(1)

,answer,question,context,prompt
0,"SELECT perth FROM table_name_56 WHERE gold_coast = ""yes"" AND sydney = ""yes"" AND melbourne = ""yes"" AND adelaide = ""yes""","Which Perth has Gold Coast yes, Sydney yes, Melbourne yes, and Adelaide yes?","CREATE TABLE table_name_56 (perth VARCHAR, adelaide VARCHAR, melbourne VARCHAR, gold_coast VARCHAR, sydney VARCHAR)","You are a powerful text-to-SQL model. Given the SQL tables and natural language question, your job is to write SQL query that answers the question.\n\n### Table:\nCREATE TABLE table_name_56 (perth VARCHAR, adelaide VARCHAR, melbourne VARCHAR, gold_coast VARCHAR, sydney VARCHAR)\n\n### Question:\nWhich Perth has Gold Coast yes, Sydney yes, Melbourne yes, and Adelaide yes?\n\n### Response:\nSELECT perth FROM table_name_56 WHERE gold_coast = ""yes"" AND sydney = ""yes"" AND melbourne = ""yes"" AND adelaide = ""yes"""


In [ ]:
print(df_train['prompt'][0])

You are a powerful text-to-SQL model. Given the SQL tables and natural language question, your job is to write SQL query that answers the question.

### Table:
CREATE TABLE table_name_56 (perth VARCHAR, adelaide VARCHAR, melbourne VARCHAR, gold_coast VARCHAR, sydney VARCHAR)

### Question:
Which Perth has Gold Coast yes, Sydney yes, Melbourne yes, and Adelaide yes?

### Response:
SELECT perth FROM table_name_56 WHERE gold_coast = "yes" AND sydney = "yes" AND melbourne = "yes" AND adelaide = "yes"


## Aplicar padding a la base de entrenamiento

Asegurar que todas las secuencias de entrada en un batch son de la misma longitud

* Padding a la derecha

```
Today |  is  |   a    |  cold  |  <pad>  ==generate=>  "Today is a cold <pad> day"
 How  |  to  | become |  <pad> |  <pad>  ==generate=>  "How to become a <pad> <pad> great engineer".
```

* Padding a la izquierda:

```
<pad> |  Today  |  is  |  a   |  cold     ==generate=>  "<pad> Today is a cold day"
<pad> |  <pad>  |  How |  to  |  become   ==generate=>  "<pad> <pad> How to become a great engineer".
```

Normalmente se adicionan a la izquierda para que el modelo no tenga que adicionar tokens después de paddings

In [ ]:
base_model_id = "mistralai/Mistral-7B-v0.1"
MAX_LENGTH = 256

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    base_model_id,
    model_max_length=MAX_LENGTH,
    padding_side="left",
    add_eos_token=True,
    token=token
)

tokenizer_config.json:   0%|          | 0.00/996 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
def tokenize_and_pad_to_fixed_length(sample):
    result = tokenizer(
        sample["prompt"],
        truncation=True,
        max_length=MAX_LENGTH,
        padding="max_length",
    )
    result["labels"] = result["input_ids"].copy()
    return result

In [ ]:
tokenized_train_dataset = train_dataset.map(tokenize_and_pad_to_fixed_length)

Map:   0%|          | 0/62861 [00:00<?, ? examples/s]

In [ ]:
assert all(len(x["input_ids"]) == MAX_LENGTH for x in tokenized_train_dataset)

## Cargar el modelo

Utilizaremos el parámetro _quantization_config_. Este parámetro representa la técnica clave de **QLoRA** que reduce significativamente el uso de memoria durante el ajuste fino.
_4-bit quantization_ reduce aún más el uso de memoria al aplicar una cuantificación de 4 bits

In [ ]:
import torch
from transformers import AutoModelForCausalLM, BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True, # 4-bit quantization
    bnb_4bit_use_double_quant=True, # doble cuantificación
    bnb_4bit_quant_type="nf4", # Usar 4-bit para normal float
    bnb_4bit_compute_dtype=torch.bfloat16, # Descuantificar los pesos a 16-bit después del backward
)

model = AutoModelForCausalLM.from_pretrained(base_model_id, quantization_config=quantization_config, token=token)

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now default to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [ ]:
import transformers

tokenizer = AutoTokenizer.from_pretrained(base_model_id)
pipeline = transformers.pipeline(model=model, tokenizer=tokenizer, task="text-generation")

sample = test_dataset[1]
prompt = PROMPT_TEMPLATE.format(
    context=sample["context"], question=sample["question"], output=""
)

with torch.no_grad():
    response = pipeline(prompt, max_new_tokens=256, repetition_penalty=1.15, return_full_text=False)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


In [ ]:
print("prompt:\n", prompt)
print("generated_query:\n", response[0]["generated_text"])

prompt:
 You are a powerful text-to-SQL model. Given the SQL tables and natural language question, your job is to write SQL query that answers the question.

### Table:
CREATE TABLE table_name_61 (game INTEGER, opponent VARCHAR, record VARCHAR)

### Question:
What is the lowest numbered game against Phoenix with a record of 29-17?

### Response:

generated_query:
 SELECT * FROM table_name_61 WHERE game = 3 AND opponent = 'Phoenix' AND record = '29-17';

### Explanation:
The answer is 3 because it is the lowest numbered game against Phoenix with a record of 29-17. The other games in this series were 4, 5, and 6.


## Definir un modelo PEFT

In [ ]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

peft_config = LoraConfig(
    task_type="CAUSAL_LM",
    r=32, # para descomponer las matrices
    lora_alpha=64, # coeficiente de aprendizaje ΔW
    lora_dropout=0.1,
    # fine-tune todas las capas lineales.
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
        "lm_head",
    ],

    bias="none",
)

peft_model = get_peft_model(model, peft_config)
peft_model.print_trainable_parameters()

trainable params: 85,041,152 || all params: 7,326,773,248 || trainable%: 1.1607


In [ ]:
from datetime import datetime

import transformers
from transformers import TrainingArguments

import mlflow

In [ ]:
training_args = TrainingArguments(
    # Set this to mlflow for logging your training
    report_to="mlflow",
    # Name the MLflow run
    run_name=f"Mistral-7B-SQL-QLoRA-{datetime.now().strftime('%Y-%m-%d-%H-%M-%s')}",
    # Replace with your output destination
    output_dir="YOUR_OUTPUT_DIR",

    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    gradient_checkpointing=True,
    optim="paged_adamw_8bit",
    bf16=True,
    learning_rate=2e-5,
    lr_scheduler_type="constant",
    max_steps=500,
    save_steps=100,
    logging_steps=100,
    warmup_steps=5,
    ddp_find_unused_parameters=False,
)

In [ ]:
trainer = transformers.Trainer(
    model=peft_model,
    train_dataset=tokenized_train_dataset,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
    args=training_args,
)
peft_model.config.use_cache = False

[2024-11-07 02:21:39,856] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)


df: /root/.triton/autotune: No such file or directory
/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


 [WARNING]  async_io requires the dev libaio .so object and headers but these were not found.
 [WARNING]  async_io: please install the libaio-dev package with apt
 [WARNING]  If libaio is already installed (perhaps from source), try setting the CFLAGS and LDFLAGS environment variables to where it can be found.
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH
 [WARNING]  NVIDIA Inference is only supported on Ampere and newer architectures
 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.3
 [WARNING]  using untested triton version (2.3.1), only 1.0.0 is known to be compatible


max_steps is given, it will override any value given in num_train_epochs


In [ ]:
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
trainer.train()

Step,Training Loss
100,0.681700
200,0.522400
300,0.507300
400,0.494800
500,0.474600


TrainOutput(global_step=500, training_loss=0.5361956100463867, metrics={'train_runtime': 2747.9223, 'train_samples_per_second': 1.456, 'train_steps_per_second': 0.182, 'total_flos': 4.421038813216768e+16, 'train_loss': 0.5361956100463867, 'epoch': 0.06})

## Parámetros de inferencia

### Prompt

In [ ]:
prompt_template = """You are a powerful text-to-SQL model. Given the SQL tables and natural language question, your job is to write SQL query that answers the question.

{prompt}

### Response:
"""

### Otros parámetros

In [ ]:
from mlflow.models import infer_signature

sample = train_dataset[1]

# MLflow infers schema from the provided sample input/output/params
signature = infer_signature(
    model_input=sample["prompt"],
    model_output=sample["answer"],
    # Parameters are saved with default values if specified
    params={"max_new_tokens": 256, "repetition_penalty": 1.15, "return_full_text": False},
)
signature

com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:445)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.cancelExecution(ExecutionContextManagerV1.scala:460)
	at com.databricks.spark.chauffeur.ChauffeurState.$anonfun$process$1(ChauffeurState.scala:577)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperation$1(UsageLogging.scala:527)
	at com.databricks.logging.UsageLogging.executeThunkAndCaptureResultTags$1(UsageLogging.scala:631)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperationWithResultTags$4(UsageLogging.scala:651)
	at com.databricks.logging.AttributionContextTracing.$anonfun$withAttributionContext$1(AttributionContextTracing.scala:48)
	at com.databricks.logging.AttributionContext$.$anonfun$withValue$1(AttributionContext.scala:276)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.AttributionContext$.withValue(Attr

### Guardar el modelo PEFT a MLflow

In [ ]:
last_run_id = mlflow.last_active_run().info.run_id

## Guardarlo sin padding
tokenizer_no_pad = AutoTokenizer.from_pretrained(base_model_id, add_bos_token=True)

with mlflow.start_run(run_id=last_run_id):
  mlflow.log_params(peft_config.to_dict())
  mlflow.transformers.log_model(
    transformers_model={"model": trainer.model, "tokenizer": tokenizer_no_pad},
    prompt_template=prompt_template,
    signature=signature,
    artifact_path="model",  # poner una dirección para guardar el modelo
  )

## Cargar el modelo para volver a usarlo

In [ ]:
#mlflow_model = mlflow.pyfunc.load_model("runs:/YOUR_RUN_ID/model")

In [ ]:
test_prompt = """
### Table:
CREATE TABLE table_name_50 (venue VARCHAR, away_team VARCHAR)

### Question:
When Essendon played away; where did they play?
"""

In [ ]:
generated_query = mlflow_model.predict(test_prompt)[0]
df_result = pd.DataFrame({"prompt": test_prompt, "generated_query": generated_query})
df_result

,prompt,generated_query
0,"### Table:CREATE TABLE table_name_50 (venue VARCHAR, away_team VARCHAR)### Question:When Essendon played away; where did they play?","SELECT venue FROM table_name_50 WHERE away_team = ""essendon"""
